# Import Your Own Data

## Import Packages

In [16]:
%reload_ext autoreload
from util.dependencies import *
from settings import USER_ID
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StructField, StructType, StringType, FloatType, TimestampType
from cerebralcortex.data_importer.data_parsers import csv_data_parser
from cerebralcortex.data_importer import import_dir

# Import Data

In [ ]:
# import CSV/IoT data (demo stream name - demo-location-data-stream)
import_dir(
    cc_config="/home/md2k/cc_conf/",
    input_data_dir="sample_data/",
    user_id=USER_ID,
    data_file_extension=[".csv"],
    data_parser=csv_data_parser,
    gen_report=True
)

## Create CerebralCortex object

In [2]:
CC = Kernel("/home/md2k/cc_conf/")

## View Imported Data

In [3]:
location_stream = CC.get_stream("iot-data-stream")
#location_stream.show(3, False)
#location_stream.plot()

## Write a smoothing algorithm

In [30]:
schema = StructType([
    StructField("ts", TimestampType())
    ])
@pandas_udf(schema, PandasUDFType.GROUPED_MAP)
def smooth_df(data):
#     df= data['some_vals'].ewm(span = 3600).mean()
#     print(type(df))
    df2 = pd.DataFrame([data.timestamp], columns=['ts'])
    return df2

## Run smoothing algorithm on imported data

In [31]:
#smooth_stream = location_stream.groupby("user").compute(smooth_df)
smooth_stream = location_stream.compute(smooth_df)
smooth_stream.show(3)

+----+
|  ts|
+----+
|null|
+----+



In [58]:
import pyspark.sql.functions
from pyspark.sql.functions import minute, second, mean, window

@pandas_udf('double', PandasUDFType.SCALAR)
def fun_function(a):
    return a+2
df=location_stream.data

#df.withColumn('d', fun_function(df.some_vals)).show()

df.groupBy("timestamp", window("timestamp", "5 minutes")).stddev("some_vals").show(5, False)

interval = 60
gdf = df.withColumn(
    'time_interval',
    pyspark.sql.functions.from_unixtime(pyspark.sql.functions.floor(pyspark.sql.functions.unix_timestamp(df['timestamp']) / interval) * interval)
).groupBy('time_interval')

#df.groupBy(second("timestamp").alias("hour")).agg(mean("some_vals").alias("mean")).show()

AttributeError: 'GroupedData' object has no attribute 'stddev'

## Visualize data

/opt/conda/lib/python3.6/site-packages/ipyleaflet/leaflet.py:80: DeprecationWarning:

Traits should be given as instances, not types (for example, `Int()`, not `Int`). Passing types is deprecated in traitlets 4.1.

/opt/conda/lib/python3.6/site-packages/ipyleaflet/leaflet.py:297: DeprecationWarning:

metadata {'dtype': None} was set from the constructor. With traitlets 4.1, metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')

/opt/conda/lib/python3.6/site-packages/ipyleaflet/leaflet.py:416: DeprecationWarning:

metadata {'trait': <traitlets.traitlets.Instance object at 0x7f33837de320>} was set from the constructor. With traitlets 4.1, metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')

/opt/conda/lib/python3.6/site-packages/ipyleaflet/leaflet.py:553: DeprecationWarning:

Traits should be given as instances, not types (for example, `Int()`, not `Int`). Passing types is deprecated in traitlets 4.1.

/opt

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …